In [1]:
import numpy as np
import numpy
import torch.nn as nn

import argparse

import os
import numpy as np
import torch
from PIL import Image

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from engine import train_one_epoch, evaluate
import utils
import transforms as T
import time

import os
import numpy as np
import torch
from PIL import Image
import torch.nn as nn

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchsummary import summary

from engine import train_one_epoch, evaluate
import utils
import transforms as T

import matplotlib.pyplot as plt
import matplotlib

matplotlib.use('TkAgg')

In [2]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [3]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model.backbone.body.conv1 = nn.Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [4]:
def load_all_label(path):
    replay_name = path.split('/')[-2]
    label_path = path.replace(replay_name + "/", '')
    label_len = len(os.listdir(label_path + replay_name +"/"))
    labels = np.array([])
    for i in range(0, label_len):
        labels = np.append(labels, np.load(label_path + replay_name +"/"+ str(i) + ".npy", allow_pickle=True)[1])

    results = labels

    return results

In [5]:
class PennFudanDataset(object):
    def __init__(self, path, transforms, window_size):
        self.root = path
        self.transforms = transforms
        pth = os.listdir(path)
        self.label_sequences = []

        self.dir_paths = []
        for i in pth:
            if os.path.isdir(path + i):
                self.label_sequences += [load_all_label(path + i +'/')]
                self.dir_paths.append(path + i + '/')
                print(f"Loaded {i}")

        self.window_size = window_size

        self.seq_indexs = []
        start = 0
        for i, seq in enumerate(self.label_sequences):
            end = start + len(seq) - (self.window_size - 1)
            self.seq_indexs.append((i, start, end))
            start = end

    def __len__(self):
        return self.seq_indexs[-1][-1]

    def __getitem__(self, idx):
        # load images and masks
        for i, start, end in self.seq_indexs:
            if idx >= start and idx < end:
                real_idx = idx - start

                data = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[0][0]    # (11, 128, 128)
                masks = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[1][0]   #  (128, 128)
                masks = np.expand_dims(masks, axis=0)
                break

        input_data = self.preprocessing(data)


        num_objs = 1
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return input_data, target


    def preprocessing(self, data):
        #0 ground 1 air 2 building 3 spell 4 ground 5 air 6 building 7 spell 8 resource 9 vision 10 terrain
        temp = np.zeros([self.window_size, 7, data.shape[2],data.shape[2]])

        temp[:,0] = data[0]
        temp[:,1] = data[1]
        temp[:,2] = data[2]
        temp[:,3] = data[4]
        temp[:,4] = data[5]
        temp[:,5] = data[6]
        temp[:,6] = data[9]

        data = temp
        data = data.reshape(self.window_size*data.shape[1],data.shape[2],-1)
        
        return torch.FloatTensor(data)    

In [6]:
import sys
data_path = f"./test_data2/"
# dataset = PennFudanDataset(data_path, get_transform(train=False), window_size=1)
dataset_test = PennFudanDataset(data_path, get_transform(train=False),window_size=1)

Loaded 36


In [7]:
len(dataset_test)

3541

In [8]:
# 수집한 데이터 중 1개에 대한 인풋, 아웃풋(정답) Plotting
index=0
img_t, target_t = dataset_test[index]
print("img_t.shape: ", img_t.shape)
img_total = img_t.sum(axis=0, keepdim=True)
plt.subplot(2,1,1)
plt.imshow(img_total[0].mul(255).byte().detach().numpy())
plt.subplot(2,1,2)
plt.imshow(target_t['masks'][0].mul(255).byte().detach().numpy().T)
plt.show()

img_t.shape:  torch.Size([7, 128, 128])


In [9]:
target_t['masks'][0]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]], dtype=torch.uint8)

In [10]:
target_t['boxes'][0]

tensor([  2., 108.,  13., 127.])

In [11]:
target_t['boxes'][0]*32

tensor([  64., 3456.,  416., 4064.])

In [61]:
# 데이터 셋 샘플링 or Plotting
# Start, End, Step = 200, 3000, 500
Start, End, Step = 0, len(dataset_test), 1
test_img_array = []
test_img_one_channel_array = []
test_target_array = []
for i in range(Start,End,Step):
    img_t, target_t = dataset_test[i]
    test_img_array.append(img_t)
    img_one_channel = img_t.sum(axis=0, keepdim=True)
    test_img_one_channel_array.append(img_one_channel)
    test_target_array.append(target_t)

In [84]:
#plot_len = int(len(test_img_array)/2)
#for i in range(plot_len):
#    plt.subplot(2,plot_len,i+1)
#    plt.imshow(test_img_one_channel_array[i][0].mul(255).byte().detach().numpy().T)
#    plt.subplot(2,plot_len,i+1+plot_len)
#    plt.imshow(test_target_array[i]['masks'][0].mul(255).byte().detach().numpy().T)
#plt.show()

In [62]:
# 학습 후 prediction 출력
num_classes = 2
model = get_model_instance_segmentation(num_classes)
model.load_state_dict(torch.load("saved_models/model_10.pth"))
model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [63]:
vpx_array = []
vpy_array = []

In [65]:
type(test_img_array)
len(test_img_array)

3541

In [79]:
with torch.no_grad():
    for idx,i in enumerate(test_img_array):
        prediction = model(torch.unsqueeze(i, 0))
        vpx = int(prediction[0]["boxes"][0][1])*32
        vpy = int(prediction[0]["boxes"][0][0])*32
        vpx_array.append(vpx)
        vpx_array.append(vpy)
        if idx %10 == 0:
            print(idx)

        
        
        

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [66]:
int(prediction[1]["boxes"][0][1])

7

In [67]:
prediction[1]["boxes"][0]

tensor([32.2523,  7.1798, 42.9717, 26.4899])

In [17]:
len(test_img_array[:10])

10

In [19]:
len(prediction)

5

In [23]:
len(prediction[0]["masks"][0])

1

In [24]:
prediction[0]["masks"][0]

tensor([[[0.1260, 0.2381, 0.2485,  ..., 0.0000, 0.0000, 0.0000],
         [0.5153, 0.9577, 0.9951,  ..., 0.0000, 0.0000, 0.0000],
         [0.5474, 0.9723, 0.9974,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]])

In [25]:
prediction[0]['masks'][0].sum(axis=0).mul(255)

tensor([[ 32.1322,  60.7049,  63.3553,  ...,   0.0000,   0.0000,   0.0000],
        [131.4081, 244.2154, 253.7569,  ...,   0.0000,   0.0000,   0.0000],
        [139.5768, 247.9257, 254.3357,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]])

In [26]:
prediction[0]["masks"][0].shape

torch.Size([1, 128, 128])

In [33]:
prediction[0]["boxes"]

tensor([[  1.2172,   0.3423,  12.0907,  19.2320],
        [113.0031, 107.1073, 124.6919, 126.2834],
        [ 10.0833,  48.6188,  21.0807,  67.7279],
        [ 15.7751,  48.7598,  26.7987,  67.5760],
        [  2.1496, 108.0664,  13.0492, 127.0174],
        [ 10.0702,  81.0988,  21.0948, 100.1526],
        [ 20.6222,  49.6542,  31.5808,  68.6506],
        [ 19.8781,  84.1622,  30.8432, 103.0791],
        [ 15.8928,  82.6276,  26.8725, 101.3527]])

In [47]:
# 인풋, 정답, prediction에 대한 Plotting
plot_len = int(len(test_img_array))
plot_len = len(test_img_array)
for i in range(plot_len):
    plt.subplot(3,plot_len,i+1)
    plt.imshow(test_img_one_channel_array[i][0].mul(255).byte().detach().numpy())
    plt.subplot(3,plot_len,i+1+ plot_len)
    plt.imshow(test_target_array[i]['masks'][0].mul(255).byte().detach().numpy().T)
    plt.subplot(3,plot_len,i+1+ plot_len*2)
    #plt.imshow(prediction[i]['masks'][3].sum(axis=0).mul(255).byte().detach().numpy().T)
    plt.imshow(prediction[i]['masks'][0].sum(axis=0).mul(255).byte().detach().numpy().T)
plt.show()

In [51]:
test_target_array[0]["boxes"]

tensor([[110., 108., 121., 127.]])

In [52]:
test_target_array[1]["boxes"]

tensor([[ 88., 105.,  99., 124.]])

In [53]:
test_target_array[2]["boxes"]

tensor([[112., 106., 123., 125.]])

In [54]:
test_target_array[3]["boxes"]

tensor([[ 85.,  97.,  96., 116.]])

In [56]:
108*32

3456

In [58]:
vpx = test_target_array[0]["boxes"][0][1]
vpy = test_target_array[0]["boxes"][0][0]

In [60]:
vpy = test_target_array[0]["boxes"][0][0]

In [ ]:
vpx_array = []
vpx_append(test_target_array[0]["boxes"][0][1])

In [48]:
test_target_array[0]["labels"]

tensor([1])

In [147]:
test_target_array[0]["masks"].max()

tensor(1, dtype=torch.uint8)

In [151]:
test_target_array[1]["boxes"][0]

tensor([110., 108., 121., 127.])

In [150]:
prediction[1]['boxes'][0]

tensor([109.4481, 108.0071, 120.7171, 127.0499])

In [129]:
test_target_array[5]["boxes"]

tensor([[ 85.,  97.,  96., 116.]])

In [114]:
target_t.keys()

dict_keys(['boxes', 'labels', 'masks', 'image_id', 'area', 'iscrowd'])

In [116]:
target_t["masks"].shape

torch.Size([1, 128, 128])

In [92]:
prediction[0]['masks'].shape

torch.Size([9, 1, 128, 128])

In [95]:
prediction[0]['labels']

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [87]:
prediction[0].keys()

dict_keys(['boxes', 'labels', 'scores', 'masks'])

In [88]:
prediction[0]["boxes"]

tensor([[ 34.7093,  18.5101,  45.5739,  37.2065],
        [ 33.0960,   6.5159,  44.1429,  26.0190],
        [  3.7773,   0.0000,  14.8168,  19.0350],
        [107.0917, 108.0264, 117.6414, 127.2897],
        [112.9700, 107.5525, 123.7498, 126.9076],
        [ 92.0005,  21.8149, 102.9863,  40.7442],
        [ 89.0254, 102.6754,  99.9427, 121.4857],
        [ 32.3551,  28.2777,  36.7931,  36.8388],
        [  2.1496, 108.0664,  13.0492, 127.0174]])

In [89]:
len(prediction[0]["boxes"])

9

In [94]:
prediction[0]['labels'].shape

torch.Size([9])

In [102]:
prediction[0]['scores']

tensor([0.9776, 0.6844, 0.4229, 0.3269, 0.1007, 0.0929, 0.0864, 0.0803, 0.0715])